In [1]:
import cadquery as cq
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults

import alphashape as alph

import numpy as np

set_defaults(default_color="#8CC5FF", axes=False, grid=True,theme="light", axes0=True, ortho=True, transparent=True)
set_sidecar("CadQuery", init=True)



Overwriting auto display for cadquery Workplane and Shape


## Bottom Plate

In [2]:
widthmm = 355.6
heightmm = 165.1
depthmm = 3.175
screwhole_diameter = 5

# base 
base = cq.Workplane('XY').box(widthmm, heightmm, depthmm)

# Corner holes
offset = 25
bottom = base.faces(">Z").workplane().rect(widthmm-offset, heightmm-offset, forConstruction=True)\
                                     .vertices().hole(screwhole_diameter)
# inner holes
bottom = bottom.faces(">Z").workplane().rect(50, heightmm-offset, forConstruction=True).vertices().hole(screwhole_diameter)



show(bottom) 

Done, using side car 'Cadquery'


## Hollow Plate

In [3]:
hollow_offset_x = 50
hollow_offset_y = 50

cavity = cq.Workplane('XY').box(widthmm-hollow_offset_x, heightmm-hollow_offset_y, depthmm)
shell = base.faces(">Z").workplane().rect(widthmm-offset, heightmm-offset, forConstruction=True)\
                                     .vertices().hole(screwhole_diameter)

shell = shell.faces(">Z").workplane().rect(50, heightmm-offset, forConstruction=True).vertices().hole(screwhole_diameter)
shell = shell.cut(cavity)
show(shell)

Done, using side car 'Cadquery'


In [4]:
right_4finger_cluster_coords = [(-26.15, -11.75), (0,0), (28.65, -3.2), (53.5, -19.45), 
                                (-26.15, -33.75), (0, -22), (28.65, -25.2), (53.5, -41.45)]
left_4finger_cluster_coords = [(-53.5, -19.45), (-28.65, -3.2), (0,0), (26.15, -11.75)]
left_thumb_cluster_coords = [(0,0), (19, -13.25)]
right_thumb_cluster_coords = [(0,0), (-19, -13.25), (0, 26.65), (-19, 13.25), (19, 13.25)] #, (-15, -15), (15, 15)]
center_options_f1_coords = [(0,0)]

f1_cap_faceplate = base.faces(">Z").center(-100, 33).pushPoints(left_4finger_cluster_coords).circle(11.25)
f1_cap_faceplate = base.faces(">Z").center(100, 55).pushPoints(right_4finger_cluster_coords).circle(11.25)
f1_cap_faceplate = base.faces(">Z").center(-70, -45).pushPoints(left_thumb_cluster_coords).circle(11.25)
f1_cap_faceplate = base.faces(">Z").center(70, -45).pushPoints(right_thumb_cluster_coords).circle(11.25)
f1_cap_faceplate = base.faces(">Z").center(0, 18).pushPoints(center_options_f1_coords).circle(11.25)
f1_cap_faceplate = f1_cap_faceplate.cutThruAll()

# show(f1_cap_faceplate)

In [5]:
keeb_cap_faceplate = base.faces(">Z").center(-100, 33).pushPoints(left_4finger_cluster_coords).rect(19, 19)
keeb_cap_faceplate = base.faces(">Z").center(100, 55).pushPoints(right_4finger_cluster_coords).rect(19, 19)
keeb_cap_faceplate = base.faces(">Z").center(-70, -45).pushPoints(left_thumb_cluster_coords).rect(19, 19)
keeb_cap_faceplate = base.faces(">Z").center(70, -45).pushPoints(right_thumb_cluster_coords).rect(19, 19)
keeb_cap_faceplate = base.faces(">Z").center(0, 18).pushPoints(center_options_f1_coords).rect(19, 19)
keeb_cap_faceplate = keeb_cap_faceplate.cutThruAll()

In [6]:
switch_plate_f1 = base.faces(">Z").center(-100, 33).pushPoints(left_4finger_cluster_coords).rect(14,14)
left_4finger_cluster_verts = switch_plate_f1.vertices().vals()
switch_plate_f1 = base.faces(">Z").center(100, 55).pushPoints(right_4finger_cluster_coords).rect(14,14)
right_4finger_cluster_verts = switch_plate_f1.vertices().vals()
switch_plate_f1 = base.faces(">Z").center(-70, -45).pushPoints(left_thumb_cluster_coords).rect(14,14)
left_thumb_cluster_verts = switch_plate_f1.vertices().vals()
switch_plate_f1 = base.faces(">Z").center(70, -45).pushPoints(right_thumb_cluster_coords).rect(14,14)
right_thumb_cluster_verts = switch_plate_f1.vertices().vals()
switch_plate_f1 = base.faces(">Z").center(0, 18).pushPoints(center_options_f1_coords).rect(14,14)
switch_plate_f1 = switch_plate_f1.cutThruAll()

# show(switch_plate_f1)

In [7]:
# vertices

In [37]:
pico_width = 21
pico_height = 53

usbc_width = 19.5
usbc_height = 44
usbc_mount_width = 28
usbc_mount_height = 9

vertices = left_4finger_cluster_verts + right_4finger_cluster_verts + left_thumb_cluster_verts + right_thumb_cluster_verts
coords = [[v.X,v.Y] for v in vertices]

clusterPolys = alph.alphashape(coords, 0.1)

usbc_slot = cq.Workplane('XY').hLine(usbc_mount_width/2).vLine(-usbc_mount_height).hLine(-(usbc_mount_width-usbc_width)/2).vLine(-1*(usbc_height-usbc_mount_height)).hLineTo(0).mirrorY()
pico_slot = cq.Workplane('XY').rect(pico_width, pico_height)

# extrude 
usbc_poly = usbc_slot.extrude(depthmm, both=True).translate((0, heightmm/2-5, 0))
pico_poly = pico_slot.extrude(depthmm, both=True).translate((0, -30, 0))

pcb_holder = base.cut(usbc_poly).cut(pico_poly)

#cut channels for wires
pcb_holder = pcb_holder.center(0, 20).rect(14, 150).center(0, -20).cutThruAll()
# cutThruAll kills the kernel, RIP
for poly in clusterPolys:
    c = cq.Workplane('XY').polyline(np.transpose([poly.boundary.xy[0], poly.boundary.xy[1]])).edges().close().extrude(depthmm, both=True)
    pcb_holder = pcb_holder.cut(c)

screwhole_diameter = 6.1
pcb_holder = pcb_holder.faces(">Z").workplane().rect(widthmm-offset, heightmm-offset, forConstruction=True)\
                                     .vertices().hole(screwhole_diameter)
pcb_holder = pcb_holder.faces(">Z").workplane().rect(50, heightmm-offset, forConstruction=True).vertices().hole(screwhole_diameter)
pcb_holder

Done, using side car 'Cadquery'


In [38]:
pcb_holder_layer2 = pcb_holder.center(0, -30).rect(154, 28).cutThruAll()
pcb_holder_layer2 = pcb_holder_layer2.center(73.85, 18).rect(14, 55).cutThruAll()
pcb_holder_layer2 = pcb_holder_layer2.center(-140, 18).rect(14, 45).cutThruAll()


In [39]:
pcb_holder_layer2

Done, using side car 'Cadquery'


In [40]:
cardboardmx = PartGroup(
    [
        # Part(cq.importers.importDXF('/mnt/chromeos/MyFiles/Downloads/3_SwitchPlate_v2.dxf').rotateAboutCenter((0,0,1), 180).translate((223.1, -181.25, 0)) ),
        Part(bottom, "bottom"),
        Part(bottom.translate((0,0,depthmm*1)), "shell"),
        Part(pcb_holder_layer2.translate((0,0,depthmm*2)), "pcb_holder2"),
        Part(pcb_holder.translate((0,0,depthmm*3)), "pcb_holder"),
        Part(switch_plate_f1.translate((0,0,depthmm*4)), "switchplate1"),
        Part(switch_plate_f1.translate((0,0,depthmm*5)), "switchplate2"),
        Part(keeb_cap_faceplate.translate((0,0,depthmm*6)), "faceplate1"),

    ],
    "carboardMX"
)

show(cardboardmx, default_color="blue")

Done, using side car 'Cadquery'


In [41]:
from cadquery import exporters

exporters.export(bottom, "artifacts/bottom.svg")
exporters.export(pcb_holder_layer2, "artifacts/pcb_layer_1.svg")
exporters.export(pcb_holder, "artifacts/pcb_layer_2.svg")
exporters.export(switch_plate_f1, "artifacts/switch_plate.svg")
exporters.export(keeb_cap_faceplate, "artifacts/keebcap_faceplate.svg")

In [42]:
# exporters.export(cardboardmx, "artifacts/controller.svg")

In [ ]:
dir()